# Introduction to [PowerSystems.jl](https://github.com/NREL-SIIP/PowerSystems.jl)

**Originally Contributed by**: Clayton Barrows and Jose Daniel Lara

## Introduction

This notebook is intended to show a power system data specification framework that exploits the
capabilities of Julia to improve performance and allow modelers to develop modular software
to create problems with different complexities and enable large scale analysis. The
[PowerSystems documentation](https://nrel-siip.github.io/PowerSystems.jl/stable/) is also
an excellent resource.

### Objective
PowerSystems.jl provides a type specification for bulk power system data.
The objective is to exploit Julia's integration of dynamic types to enable efficient data
handling and enable functional dispatch in modeling and analysis applications
As explained in Julia's documentation:

"Julia’s type system is dynamic, but gains some of the advantages of static type systems
by making it possible to indicate that certain values are of specific types. This can be
of great assistance in generating efficient code, but even more significantly, it allows
method dispatch on the types of function arguments to be deeply integrated with the language."

For more details on Julia types, refer to the [documentation](https://docs.julialang.org/en/v1/)


## Environment and packages

PowerSystems.jl relies on a framework for data handling established in
[InfrastructureSystems.jl](https://github.com/NREL-SIIP/InfrastructureSystems.jl).
Users of PowerSystems.jl should not need to interact directly with InfrastructureSystems.jl.
However, it's worth recognizing that InfrastructureSystems provides much of the back end
code for managing and accessing data, especially time series data.

In [1]:
using SIIPExamples;
using PowerSystems;
using D3TypeTrees;
IS = PowerSystems.IS

InfrastructureSystems

## Types in PowerSystems
PowerSystems.jl provides a type hierarchy for specifying power system data. Data that
describes infrastructure components is held in `struct`s. For example, a `Bus` is defined
as follows with fields for the parameters required to describe a bus (along with an
`internal` field used by InfrastructureSystems to improve the efficiency of handling data).

In [2]:
print_struct(Bus)

mutable struct Bus
    number::Int64
    name::String
    bustype::Union{Nothing, PowerSystems.BusTypes.BusType}
    angle::Union{Nothing, Float64}
    magnitude::Union{Nothing, Float64}
    voltage_limits::Union{Nothing, NamedTuple{(:min, :max),Tuple{Float64,Float64}}}
    base_voltage::Union{Nothing, Float64}
    area::Union{Nothing, Area}
    load_zone::Union{Nothing, LoadZone}
    ext::Dict{String,Any}
    internal::InfrastructureSystems.InfrastructureSystemsInternal
end


### Type Hierarchy
PowerSystems is intended to organize data containers by the behavior of the devices that
the data represents. To that end, a type hierarchy has been defined with several levels of
abstract types starting with `InfrastructureSystemsType`. There are a bunch of subtypes of
`InfrastructureSystemsType`, but the important ones to know about are:
- `Component`: includes all elements of power system data
  - `Topology`: includes non physical elements describing network connectivity
  - `Service`: includes descriptions of system requirements (other than energy balance)
  - `Device`: includes descriptions of all the physical devices in a power system
- `InfrastructureSystems.DeviceParameter`: includes structs that hold data describing the
 dynamic, or economic capabilities of `Device`.
- `TimeSeriesData`: Includes all time series types
  - `time series`: includes structs to define time series of forecasted data where multiple
values can represent each time stamp
  - `StaticTimeSeries`: includes structs to define time series with a single value for each
time stamp
- `System`: collects all of the `Component`s

*The following trees are made with [D3TypeTrees](https://github.com/claytonpbarrows/D3TypeTrees.jl),
nodes that represent Structs will show the Fields in the hoverover tooltip.*

TypeTree(InfrastructureSystemsType)

### `TimeSeriesData`
[_Read the Docs!_](https://nrel-siip.github.io/PowerSystems.jl/stable/modeler_guide/time_series/)
Every `Component` has a `time_series_container::InfrastructureSystems.TimeSeriesContainer`
field. `TimeSeriesData` are used to hold time series information that describes the
temporally dependent data of fields within the same struct. For example, the
`ThermalStandard.time_series_container` field can
describe other fields in the struct (`available`, `activepower`, `reactivepower`).

`TimeSeriesData`s themselves can take the form of the following:

In [3]:
TypeTree(TimeSeriesData)

TimeSeriesData
├──Forecast
│  ├──AbstractDeterministic
│  │  ├──DeterministicSingleTimeSeries (0 children)
│  │  └──Deterministic (0 children)
│  ├──Scenarios
│  └──Probabilistic
└──StaticTimeSeries
   └──SingleTimeSeries

In each case, the time series contains fields for `scaling_factor_multiplier` and `data`
to identify the details of  th `Component` field that the time series describes, and the
time series `data`. For example: we commonly want to use a time series to
describe the maximum active power capability of a renewable generator. In this case, we
can create a `SingleTimeSeries` with a `TimeArray` and an accessor function to the
maximum active power field in the struct describing the generator. In this way, we can
store a scaling factor time series that will get multiplied by the maximum active power
rather than the magnitudes of the maximum active power time series.

In [4]:
print_struct(Deterministic)

mutable struct Deterministic
    name::String
    data::Union{DataStructures.SortedDict{DateTime,Array{Array{Tuple{Float64,Float64},1},1},Ord} where Ord<:Base.Order.Ordering, DataStructures.SortedDict{DateTime,Array{Float64,1},Ord} where Ord<:Base.Order.Ordering, DataStructures.SortedDict{DateTime,Array{Tuple{Float64,Float64},1},Ord} where Ord<:Base.Order.Ordering}
    resolution::Period
    scaling_factor_multiplier::Union{Nothing, Function}
    internal::InfrastructureSystems.InfrastructureSystemsInternal
end


Examples of how to create and add time series to system can be found in the
[Add Time Series Example](../PowerSystems.jl Examples/add_forecasts.ipynb)

### System
The `System` object collects all of the individual components into a single struct along
with some metadata about the system itself (e.g. `base_power`)

In [5]:
print_struct(System)

 struct System
    data::InfrastructureSystems.SystemData
    frequency::Float64
    bus_numbers::Set{Int64}
    runchecks::Bool
    units_settings::InfrastructureSystems.SystemUnitsSettings
    internal::InfrastructureSystems.InfrastructureSystemsInternal
end


## Basic example
PowerSystems contains a few basic data files (mostly for testing and demonstration).

In [6]:
BASE_DIR = abspath(joinpath(dirname(Base.find_package("PowerSystems")), ".."))
include(joinpath(BASE_DIR, "test", "data_5bus_pu.jl")) #.jl file containing 5-bus system data
nodes_5 = nodes5() # function to create 5-bus buses

5-element Array{Bus,1}:
 Bus(1, nodeA, PV, 0.0, 1.0, (min = 0.9, max = 1.05), 230.0, nothing, nothing, Dict{String,Any}())
 Bus(2, nodeB, PQ, 0.0, 1.0, (min = 0.9, max = 1.05), 230.0, nothing, nothing, Dict{String,Any}())
 Bus(3, nodeC, PV, 0.0, 1.0, (min = 0.9, max = 1.05), 230.0, nothing, nothing, Dict{String,Any}())
 Bus(4, nodeD, REF, 0.0, 1.0, (min = 0.9, max = 1.05), 230.0, nothing, nothing, Dict{String,Any}())
 Bus(5, nodeE, PV, 0.0, 1.0, (min = 0.9, max = 1.05), 230.0, nothing, nothing, Dict{String,Any}())

### Create a `System`

In [7]:
sys = System(
    100.0,
    nodes_5,
    vcat(thermal_generators5(nodes_5), renewable_generators5(nodes_5)),
    loads5(nodes_5),
    branches5(nodes_5),
)

,ConcreteType,SuperTypes,Count
,String,String,Int64
1,Arc,Topology <: Component <: InfrastructureSystemsComponent <: InfrastructureSystemsType <: Any,6
2,Bus,Topology <: Component <: InfrastructureSystemsComponent <: InfrastructureSystemsType <: Any,5
3,Line,ACBranch <: Branch <: Device <: Component <: InfrastructureSystemsComponent <: InfrastructureSystemsType <: Any,6
4,PowerLoad,StaticLoad <: ElectricLoad <: StaticInjection <: Device <: Component <: InfrastructureSystemsComponent <: InfrastructureSystemsType <: Any,3
5,RenewableDispatch,RenewableGen <: Generator <: StaticInjection <: Device <: Component <: InfrastructureSystemsComponent <: InfrastructureSystemsType <: Any,3
6,ThermalStandard,ThermalGen <: Generator <: StaticInjection <: Device <: Component <: InfrastructureSystemsComponent <: InfrastructureSystemsType <: Any,5


### Accessing `System` Data
PowerSystems provides functional interfaces to all data. The following examples outline
the intended approach to accessing data expressed using PowerSystems.

PowerSystems enforces unique `name` fields between components of a particular concrete type.
So, in order to retrieve a specific component, the user must specify the type of the component
along with the name and system

#### Accessing components

In [8]:
@show get_component(Bus, sys, "nodeA")
@show get_component(Line, sys, "1")

get_component(Bus, sys, "nodeA") = Bus(1, nodeA, PV, 0.0, 1.0, (min = 0.9, max = 1.05), 230.0, nothing, nothing, Dict{String,Any}())
get_component(Line, sys, "1") = Line(1, true, 0.0, 0.0, Arc(Bus(1, nodeA, PV, 0.0, 1.0, (min = 0.9, max = 1.05), 230.0, nothing, nothing, Dict{String,Any}()), Bus(2, nodeB, PQ, 0.0, 1.0, (min = 0.9, max = 1.05), 230.0, nothing, nothing, Dict{String,Any}())), 0.00281, 0.0281, (from = 0.00356, to = 0.00356), 2.0, (min = -0.7, max = 0.7), Service[], Dict{String,Any}())


1 (Line):
   name: 1
   available: true
   active_power_flow: 0.0
   reactive_power_flow: 0.0
   arc: nodeA -> nodeB (Arc)
   r: 0.00281
   x: 0.0281
   b: (from = 0.00356, to = 0.00356)
   rate: 2.0
   angle_limits: (min = -0.7, max = 0.7)
   services: 0-element Array{Service,1}
   ext: Dict{String,Any}()
   time_series_container: InfrastructureSystems.TimeSeriesContainer: 0
   InfrastructureSystems.SystemUnitsSettings:
      base_value: 100.0
      unit_system: SYSTEM_BASE

Similarly, you can access all the components of a particular type: *note: the return type
of get_components is a `FlattenIteratorWrapper`, so call `collect` to get an `Array`

In [9]:
get_components(Bus, sys) |> collect

5-element Array{Bus,1}:
 Bus(1, nodeA, PV, 0.0, 1.0, (min = 0.9, max = 1.05), 230.0, nothing, nothing, Dict{String,Any}())
 Bus(3, nodeC, PV, 0.0, 1.0, (min = 0.9, max = 1.05), 230.0, nothing, nothing, Dict{String,Any}())
 Bus(5, nodeE, PV, 0.0, 1.0, (min = 0.9, max = 1.05), 230.0, nothing, nothing, Dict{String,Any}())
 Bus(2, nodeB, PQ, 0.0, 1.0, (min = 0.9, max = 1.05), 230.0, nothing, nothing, Dict{String,Any}())
 Bus(4, nodeD, REF, 0.0, 1.0, (min = 0.9, max = 1.05), 230.0, nothing, nothing, Dict{String,Any}())

`get_components` also works on abstract types:

In [10]:
get_components(Branch, sys) |> collect

6-element Array{Branch,1}:
 Line(4, true, 0.0, 0.0, Arc(Bus(2, nodeB, PQ, 0.0, 1.0, (min = 0.9, max = 1.05), 230.0, nothing, nothing, Dict{String,Any}()), Bus(3, nodeC, PV, 0.0, 1.0, (min = 0.9, max = 1.05), 230.0, nothing, nothing, Dict{String,Any}())), 0.00108, 0.0108, (from = 0.00926, to = 0.00926), 11.148, (min = -0.7, max = 0.7), Service[], Dict{String,Any}())
 Line(1, true, 0.0, 0.0, Arc(Bus(1, nodeA, PV, 0.0, 1.0, (min = 0.9, max = 1.05), 230.0, nothing, nothing, Dict{String,Any}()), Bus(2, nodeB, PQ, 0.0, 1.0, (min = 0.9, max = 1.05), 230.0, nothing, nothing, Dict{String,Any}())), 0.00281, 0.0281, (from = 0.00356, to = 0.00356), 2.0, (min = -0.7, max = 0.7), Service[], Dict{String,Any}())
 Line(5, true, 0.0, 0.0, Arc(Bus(3, nodeC, PV, 0.0, 1.0, (min = 0.9, max = 1.05), 230.0, nothing, nothing, Dict{String,Any}()), Bus(4, nodeD, REF, 0.0, 1.0, (min = 0.9, max = 1.05), 230.0, nothing, nothing, Dict{String,Any}())), 0.00297, 0.0297, (from = 0.00337, to = 0.00337), 40.53, (min = -0

The fields within a component can be accessed using the `get_*` functions:
*It's highly recommended that users avoid using the `.` to access fields since we make no
guarantees on the stability field names and locations. We do however promise to keep the
accessor functions stable.*

In [11]:
bus1 = get_component(Bus, sys, "nodeA")
@show get_name(bus1);
@show get_magnitude(bus1);

get_name(bus1) = "nodeA"
get_magnitude(bus1) = 1.0


#### Accessing `TimeSeries`

First we need to add some time series to the `System`

In [12]:
loads = collect(get_components(PowerLoad, sys))
for (l, ts) in zip(loads, load_timeseries_DA[2])
    add_time_series!(
        sys,
        l,
        Deterministic(
            "activepower",
            Dict(TimeSeries.timestamp(load_timeseries_DA[2][1])[1] => ts),
        ),
    )
end

If we want to access a specific time series for a specific component, we need to specify:
 - time series type
 - `component`
 - initial_time
 - label

We can find the initial time of all the time series in the system:

In [13]:
get_forecast_initial_times(sys)

DateTime("2024-01-02T00:00:00"):Second(1):DateTime("2024-01-02T00:00:00")

We can find the names of all time series attached to a component:

In [14]:
ts_names = get_time_series_names(Deterministic, loads[1])

1-element Array{String,1}:
 "activepower"

We can access a specific time series for a specific component:

In [15]:
ta = get_time_series_array(Deterministic, loads[1], ts_names[1])

24×1 TimeSeries.TimeArray{Float64,1,DateTime,SubArray{Float64,1,Array{Float64,1},Tuple{UnitRange{Int64}},true}} 2024-01-02T00:00:00 to 2024-01-02T23:00:00
│                     │ A      │
├─────────────────────┼────────┤
│ 2024-01-02T00:00:00 │ 0.7948 │
│ 2024-01-02T01:00:00 │ 0.7807 │
│ 2024-01-02T02:00:00 │ 0.7768 │
│ 2024-01-02T03:00:00 │ 0.6945 │
│ 2024-01-02T04:00:00 │ 0.7237 │
│ 2024-01-02T05:00:00 │ 0.742  │
│ 2024-01-02T06:00:00 │ 0.6983 │
│ 2024-01-02T07:00:00 │ 0.7826 │
│ 2024-01-02T08:00:00 │ 0.7832 │
   ⋮
│ 2024-01-02T16:00:00 │ 0.9184 │
│ 2024-01-02T17:00:00 │ 0.995  │
│ 2024-01-02T18:00:00 │ 1.0509 │
│ 2024-01-02T19:00:00 │ 1.0187 │
│ 2024-01-02T20:00:00 │ 1.0866 │
│ 2024-01-02T21:00:00 │ 1.0399 │
│ 2024-01-02T22:00:00 │ 0.9793 │
│ 2024-01-02T23:00:00 │ 0.8974 │

Or, we can just get the values of the time series:

In [16]:
ts = get_time_series_values(Deterministic, loads[1], ts_names[1])

24-element view(::Array{Float64,1}, 1:24) with eltype Float64:
 0.7947960526897276
 0.780703068632554
 0.776829280744565
 0.6945199778339252
 0.7236886705967136
 0.7420467718833068
 0.6982652539188448
 0.7825798937714933
 0.7832240046033633
 0.8479716933499408
 ⋮
 0.8460206966081154
 0.9183628697994863
 0.9950118922363824
 1.050937802442711
 1.018749037924968
 1.0866361095545929
 1.0399008072140905
 0.9792855031225197
 0.897411028410693

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*